#### Create Embeddings for CDR3 Sequences (with specified model)
##### Edit input and outputs paths. Input path should direct to the csv that has the sequences to create the embeddings for, and the output path should direct to the csv that will hold the embeddings.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/ubuntu/CVC


#### Set Environment

In [3]:
import os, sys
import pandas as pd
SRC_DIR = "cvc"
assert os.path.isdir(SRC_DIR), f"Cannot find src dir: {SRC_DIR}"
sys.path.append(SRC_DIR)

from cvc import model_utils

from lab_notebooks.utils import SC_TRANSFORMER, TRANSFORMER, DEVICE
MODEL_DIR = os.path.join(SRC_DIR, "models")
sys.path.append(MODEL_DIR)

FILT_EDIT_DIST = True

2023-09-03 11:48:01.490444: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-03 11:48:01.492158: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 11:48:01.529620: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 11:48:01.530551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 11:48:02.106657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

#### Specify Parameters

In [15]:
# the sequences in the csv file need to be in a column called Sequences
input_path = "/home/ubuntu/RandomData/data/dfA_CDR3s.csv"
output_path = "/home/ubuntu/RandomData/data/dfA_CDR3s_romi_embeddings.csv"
label_column = ""

In [23]:
from lab_notebooks.utils import HOME_DIR_GCP
HOME_DIR_GCP

'/home/ubuntu/CVC'

In [16]:
# specify which model to use
# CVC - TRANSFORMER
# scCVC - SC_TRANSFORMER
TRANSFORMER_TO_USE = SC_TRANSFORMER

#### Load Data

In [17]:
# load data
tcrb_data = pd.read_csv(input_path, index_col=0)
# drop duplicates
tcrb_data = tcrb_data.drop_duplicates()
# drop rows with NaN
tcrb_data = tcrb_data.dropna()

In [18]:
# make tcrb_data a dataframe with Sequences column
tcrb_data = pd.DataFrame(tcrb_data)

In [19]:
tcrb_data

,CDR3_A,pool1_S1_TRA_mig_cdr3_clones_all,pool1_S10_TRA_mig_cdr3_clones_all,pool1_S11_TRA_mig_cdr3_clones_all,pool1_S12_TRA_mig_cdr3_clones_all,pool1_S13_TRA_mig_cdr3_clones_all,pool1_S14_TRA_mig_cdr3_clones_all,pool1_S15_TRA_mig_cdr3_clones_all,pool1_S16_TRA_mig_cdr3_clones_all,pool1_S17_TRA_mig_cdr3_clones_all,...,pool9_S22_TRA_mig_cdr3_clones_all,pool9_S23_TRA_mig_cdr3_clones_all,pool9_S24_TRA_mig_cdr3_clones_all,pool9_S3_TRA_mig_cdr3_clones_all,pool9_S4_TRA_mig_cdr3_clones_all,pool9_S5_TRA_mig_cdr3_clones_all,pool9_S6_TRA_mig_cdr3_clones_all,pool9_S7_TRA_mig_cdr3_clones_all,pool9_S8_TRA_mig_cdr3_clones_all,pool9_S9_TRA_mig_cdr3_clones_all
0,CAVNTGGFKTIF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,CAVNTGFQKLVF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,CAENSGGSNYKLTF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0
3,CAVNSGGYQKVTF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,CAVNQAGTALIF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015382,YYTPLVF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2015383,YYTTNYGNNRLAF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2015384,YYYDKVIF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2015385,YYYDTGNQFYF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [9]:
tcrb_data

In [20]:
# rename sequences column if not named "Sequences"
tcrb_data.rename(columns={'CDR3_A': 'Sequences'}, inplace=True)

In [25]:
tcrb_data

,Sequences,pool1_S1_TRA_mig_cdr3_clones_all,pool1_S10_TRA_mig_cdr3_clones_all,pool1_S11_TRA_mig_cdr3_clones_all,pool1_S12_TRA_mig_cdr3_clones_all,pool1_S13_TRA_mig_cdr3_clones_all,pool1_S14_TRA_mig_cdr3_clones_all,pool1_S15_TRA_mig_cdr3_clones_all,pool1_S16_TRA_mig_cdr3_clones_all,pool1_S17_TRA_mig_cdr3_clones_all,...,pool9_S22_TRA_mig_cdr3_clones_all,pool9_S23_TRA_mig_cdr3_clones_all,pool9_S24_TRA_mig_cdr3_clones_all,pool9_S3_TRA_mig_cdr3_clones_all,pool9_S4_TRA_mig_cdr3_clones_all,pool9_S5_TRA_mig_cdr3_clones_all,pool9_S6_TRA_mig_cdr3_clones_all,pool9_S7_TRA_mig_cdr3_clones_all,pool9_S8_TRA_mig_cdr3_clones_all,pool9_S9_TRA_mig_cdr3_clones_all
0,CAVNTGGFKTIF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,CAVNTGFQKLVF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,CAENSGGSNYKLTF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0
3,CAVNSGGYQKVTF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,CAVNQAGTALIF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015382,YYTPLVF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2015383,YYTTNYGNNRLAF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2015384,YYYDKVIF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2015385,YYYDTGNQFYF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


#### Create embeddings

In [26]:
from cvc.embbeding_wrapper import EmbeddingWrapper

# Create embeddings
embed_wrap = EmbeddingWrapper(TRANSFORMER_TO_USE, DEVICE, tcrb_data, batch_size=256, method="mean", layers=[-1], pbar=True, max_len=120)
embed_wrap.embeddings.shape

Some weights of the model checkpoint at /home/ubuntu/CVC/output_dir_singlecell_v2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/7873 [00:00<?, ?it/s]

(2015387, 768)

In [27]:
tcrb_embeddings_df = pd.DataFrame(embed_wrap.embeddings)
tcrb_embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.163495,0.080760,0.476696,-0.579615,0.884860,-0.195561,0.275461,-0.403593,0.178176,-0.486899,...,-0.055365,0.034403,-0.081032,0.213817,-0.141227,0.420788,0.368493,-0.272782,-0.849871,0.086445
1,-0.022704,-0.812257,0.003812,-0.729871,0.204372,-0.404436,0.677289,-0.233194,0.275772,-0.729323,...,-0.362977,-0.591385,-0.487568,-0.242484,-0.240120,1.246368,0.101414,0.029819,-0.484888,1.041232
2,0.798835,0.413719,0.911860,-0.133170,0.403559,-0.780708,-0.368413,-0.211868,0.210744,-0.901089,...,0.105163,-0.705243,0.084560,0.062431,-1.204412,-0.361698,0.049612,-0.489227,-1.033516,0.400039
3,0.229236,0.292643,0.960917,-0.008623,0.367363,0.295858,0.188792,0.084871,0.550106,-0.515812,...,-0.147336,-0.502519,-0.160751,0.339673,-0.486529,0.251945,-0.145595,0.371871,-0.457824,-0.136663
4,0.415509,0.290809,0.747348,-0.488818,0.169982,-0.781476,0.221385,-0.375270,-0.283548,-0.465843,...,0.399415,-0.662445,-1.347678,0.296099,0.111350,0.795811,-0.220372,0.261410,0.487527,0.904114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015382,0.569057,0.162600,0.089978,0.012106,0.419535,-0.437577,1.252888,0.310122,-0.400553,0.147231,...,-0.468567,-0.721760,0.135769,0.089239,0.188486,0.462668,-0.484096,-0.257011,-0.352276,0.669514
2015383,0.494744,0.378554,0.620226,0.349289,0.515694,-0.605529,0.424776,-0.101742,-0.168120,0.029810,...,0.070515,-1.278155,-0.629228,-0.091008,-0.828578,0.243136,-0.368765,-0.671473,-0.222074,0.492664
2015384,0.857174,0.218169,-0.012172,-0.279211,0.405902,-0.330954,0.568388,-0.368292,0.436815,0.229530,...,0.840342,-0.411674,0.391742,0.576176,0.207879,0.138552,0.521715,-0.284631,-0.996801,-0.010976
2015385,0.150761,0.100043,0.952118,0.091272,0.221874,-0.652297,-0.132203,0.724841,-0.980853,-0.600392,...,-0.349036,0.098570,-0.304584,-0.772568,-0.436647,0.262792,0.007044,0.313282,-0.322594,0.079108


In [ ]:
# create anndata object
tcrb_embeddings_adata = embed_wrap.create_anndata()

# add public/private label to dataframe
df_embeddings_with_label = tcrb_embeddings_df
df_embeddings_with_label['Sequences']=list(tcrb_embeddings_adata.obs['Sequences'])
df_embeddings_with_label[label_column]=list(tcrb_embeddings_adata.obs[label_column])
df_embeddings_with_label

In [28]:
OUTPUT_FORMAT = "pickle" # csv or pickle

In [29]:
# output embeddings to csv
if OUTPUT_FORMAT == "csv":
    tcrb_embeddings_df.to_csv(output_path, index=False)
elif OUTPUT_FORMAT == "pickle":
    import pickle
    with open(output_path, 'wb') as handle:
        pickle.dump(tcrb_embeddings_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
# save anndata object
tcrb_embeddings_adata.write_h5ad(output_path.replace(".csv", ".h5ad"))

In [32]:
with open("/home/ubuntu/RandomData/data/dfB_CDR3s_romi_embeddings.csv", 'rb') as handle:
     dfB = pickle.load(handle)